# DES light curves fitting


### 1. Imports 


In [22]:
from astropy.io import fits
import pandas as pd
import matplotlib.pyplot as plt
from astropy.table import Table
from numpy import log10 as log10
from astropy.table import Table
import astropy.units as u
import numpy as np
import os
import csv
import numpy as np
import numpy
import pandas as pd
import parsnip
import lcdata
import sncosmo
from matplotlib import pyplot as plt
from numpy import *
import scipy.optimize as opt
from chainconsumer import ChainConsumer
import emcee
import numpy as np
from astropy.table import Table
from pandas import DataFrame
import scipy.stats as st
import corner
from mpfit import mpfit
#import mpfit
from scipy.interpolate import interp1d
import math
from numpy import sqrt as sqrt
from numpy import square as square
import random
import pandas as pd
import warnings
warnings.filterwarnings("ignore")




### 2. Data Configuration and Preprocessing




In [23]:
tr=3
t1=4
countshow=0


color=['cyan','green', 'red',  'blue', 'm', 'black']
dict_color={ 'g':color[1], 'r':color[2], 'i':color[3], 'z':color[4]}
dict_number={ 'g':3.0, 'r':1.0, 'i':-1.0, 'z':-3.0}
dict_sym={'g':'s', 'r':'*', 'i':'o', 'z':'^'}
dict_legend={ 'g':'g+3.0', 'r':'r+1.0', 'i':'i-1.0', 'z':'z-3.0'}
dict_ms={'g':40, 'r':75, 'i':45, 'z':60}

data_3=[]
table_attm3=[]



phot_df = Table.read('lcs.FITS').to_pandas()
head_df = Table.read('meta.FITS').to_pandas()
mapping_coeff=pd.read_csv('mapping_coeff_original.csv')
mapping_coeff = mapping_coeff.set_index('Filter')

print("Number of photometry datapoints:", len(phot_df))
print("Number of transients:", len(head_df))

eigen_val=np.genfromtxt('eigen_val.txt')

ph=eigen_val[1:,0]
vec0=eigen_val[1:,1]
vec1=eigen_val[1:,2]
vec2=eigen_val[1:,3]


imod0 = interp1d(ph, vec0, fill_value='extrapolate')
imod1 = interp1d(ph, vec1, fill_value='extrapolate')
imod2 = interp1d(ph, vec2, fill_value='extrapolate')

os.makedirs('tables', exist_ok=True)
os.makedirs('plots', exist_ok=True)




Number of photometry datapoints: 1798736
Number of transients: 19706


### 4. Model and Utility Functions

In [24]:
def leng_calc1(time_array2, t_o2):
    mphase = (time_array2-t_o2)/(1+redshift_p)
    indsel=(np.where((mphase>=-10.0) & (mphase<=40.0)))[0]
    return len(indsel)


def modelcurve1(x_calc, theta, arr_pass):
    t_min, mag_o, a1, a2 = theta
    mphase = (x_calc-t_min)/(1+redshift_p)

    if mphase >= -10.0 and mphase <= 40.0:
        VecP0 = float(imod0(mphase))
        VecP1 = float(imod1(mphase))
        VecP2 = float(imod2(mphase))

        y_calc = mag_o + VecP0 + a1*VecP1 + a2*VecP2
       
        err_pc_scores=np.square(arr_pass[0]* VecP1)+np.square(arr_pass[1]*VecP2)
      
        
        err_cal= err_pc_scores
      
    else:
        y_calc=np.nan
        err_cal=np.nan
          
    return y_calc, err_cal



def myfunctlin1(p, fjac=None, x=None, y=None, err=None, arr_pass=None, ini_params=None, priors=None):
    
    dev = np.zeros((len(x)), dtype = np.float64)
    pr_arr = np.zeros((2), dtype = np.float64)
   
    
    for kk in range(len(dev)):
        
        y_calc, tem=modelcurve1(x[kk], p, arr_pass)
        
        if not np.isnan(y_calc):
            dev[kk]=(y[kk]-y_calc)/sqrt(np.square(err[kk])+ tem)
    
    pr_arr[0]=(p[2]-priors[0])/(sqrt(2)*arr_pass[0])
    pr_arr[1]=(p[3]-priors[1])/(sqrt(2)*arr_pass[1])


    dev_f=np.concatenate((dev, pr_arr))   
    #print(len(dev_f))    
    status = 0
    # print('dev')
    # print(len(dev_f))
    return [status, dev_f]


def mpfit_run_flux1(time_array, data_vec, data_vec_unc, ini_params, tym_cons_low, tym_cons_high, arr_pass, priors, redshift_p):

    leng=leng_calc1(time_array, ini_params[0]) 
    #print('leng')
    #print(leng)
    if(leng<4):
        
        return np.zeros((4)), np.zeros((4)), 10000, 0
    p0=ini_params
   
    priors=priors
    parbase={'value':0., 'fixed':0, 'limited':[0,0], 'limits':[0.,0.]}
    parinfo=[]
    for i in range(len(p0)):
        parinfo.append({'value':0., 'fixed':0, 'limited':[0,0], 'limits':[0.,0.]})
    for i in range(len(p0)): 
        parinfo[i]['value']=p0[i]

    parinfo[0]['limited'][0] = 1
    parinfo[0]['limits'][0]  = tym_cons_low
    parinfo[0]['limited'][1] = 1
    parinfo[0]['limits'][1]  = tym_cons_high


    parinfo[2]['limited'][0] = 1
    parinfo[2]['limits'][0]  = -10
    parinfo[2]['limited'][1] = 1
    parinfo[2]['limits'][1]  = 10

    parinfo[3]['limited'][0] = 1
    parinfo[3]['limits'][0]  = -10
    parinfo[3]['limited'][1] = 1
    parinfo[3]['limits'][1]  = 10


    fa = {'x':time_array, 'y':data_vec, 'err':data_vec_unc, 'arr_pass': arr_pass, 'ini_params':ini_params, 'priors': priors}
    
    m = mpfit(myfunctlin1,  parinfo=parinfo, quiet=True, functkw=fa)

    if (m.status <= 0): 
        #print( 'error message = ', m.errmsg)
        am=0.0
    else:
        am=m.fnorm

    #print(m.params, m.perror, am)
    return m.params, m.perror, am, leng-2


print('yes')


yes


### Main loop over light curves

In [25]:
for ii in range(len(head_df)):
    if((ii%100)==0):
        print(f"ii={ii}")
    if(head_df['SNTYPE'].iloc[ii]==0 or head_df['SNTYPE'].iloc[ii]==5 or head_df['SNTYPE'].iloc[ii]>100):
        continue
    if(head_df['REDSHIFT_HELIO'].iloc[ii]<0) or (head_df['FAKE'].iloc[ii]!=0):
        continue
   
    data_1=[]
    
    row = head_df.iloc[ii]
  
    start = int(row['PTROBS_MIN'])
    end = int(row['PTROBS_MAX']) + 1  # inclusive index

    
    table_attm1=[]
    
    countshow=countshow+1
   
    type=head_df['SNTYPE'].iloc[ii]
    obj_id=head_df['SNID'].iloc[ii]
    obj_id=int(obj_id.decode().strip())

    redshift_p=head_df['REDSHIFT_HELIO'].iloc[ii]
    #print(redshift_p)
   
    redshift_e=head_df['REDSHIFT_HELIO_ERR'].iloc[ii]
    lc = phot_df.iloc[start:end].copy()

    g=[]
    r=[]
    i=[]
    z=[]
    
    gerr=[]
    rerr=[] 
    ierr=[]
    zerr=[]
  
    g_t=[]
    r_t=[]
    i_t=[]
    z_t=[]
  
    count=0
    fil=[]
    array_mag=[]
    array_time=[]
    array_err=[]

    lc['BAND'] = lc['BAND'].apply(lambda x: x.decode('utf-8').strip() if isinstance(x, bytes) else str(x).strip())
   
    # mag1=(-2.5*log10(lc['FLUXCAL'])+lc['ZEROPT']).to_numpy()

    mag1=(-2.5*log10(lc['FLUXCAL'])+27.5).to_numpy()
    mag_err1=abs(1.086*lc['FLUXCALERR']/lc['FLUXCAL']).to_numpy()
    filter= lc['BAND'].to_numpy()
    time=lc['MJD'].to_numpy()

    mask=(~np.isnan(mag1)) & (mag_err1 < 0.5)
    mag1 = mag1[mask]
    mag_err1 = mag_err1[mask]
    filter = filter[mask]
    time = time[mask]

    for jj in range(len(mag1)):
         if(filter[jj]=='g'):
            g.append(mag1[jj])
            gerr.append(mag_err1[jj])
            g_t.append(time[jj])

         elif(filter[jj]=='r'):
            r.append(mag1[jj])
            rerr.append(mag_err1[jj])
            r_t.append(time[jj])

         elif(filter[jj]=='i'):
            i.append(mag1[jj])
            ierr.append(mag_err1[jj])
            i_t.append(time[jj])

         elif(filter[jj]=='z'):
            z.append(mag1[jj])
            zerr.append(mag_err1[jj])
            z_t.append(time[jj])
            
        
    if(g):
        g=np.array(g)
        if(len(g)>t1):
            count=count+1
            gerr=np.array(gerr)
            g_t=np.array(g_t)
            array_mag.append(g)
            array_time.append(g_t)
            array_err.append(gerr)
            fil.append('g')
        
        

    if(r):
        r=np.array(r)
        if(len(r)>t1):
            count=count+1
            rerr=np.array(rerr)
            r_t=np.array(r_t) 
            array_mag.append(r)
            array_time.append(r_t)
            array_err.append(rerr) 
            fil.append('r')

        
    if(i):
        i=np.array(i)
        if(len(i)>t1):
            count=count+1
            ierr=np.array(ierr)
            i_t=np.array(i_t) 
            array_mag.append(i)
            array_time.append(i_t)
            array_err.append(ierr)
            fil.append('i')

    if(z):
        z=np.array(z)
        if(len(i)>t1):
            count=count+1
            zerr=np.array(zerr)
            z_t=np.array(z_t) 
            array_mag.append(z)
            array_time.append(z_t)
            array_err.append(zerr)
            fil.append('z')

    if(count>0):

    
        a_arr=np.zeros((len(array_mag)))
        for kkk in range(len(array_mag)):
            
            a_arr[kkk]=len(array_mag[kkk])

        b_arr=np.argsort(-a_arr)

        fil = [fil[ipip] for ipip in b_arr]
    
        array_mag = [array_mag[ipip] for ipip in b_arr]
        array_time = [array_time[ipip] for ipip in b_arr]

        array_err = [array_err[ipip] for ipip in b_arr]
        # print('count')
        # print(count)
        if((array_time[0][0]-head_df['PEAKMJD'].iloc[ii])<10):
        

            for momo in range(count):

                    
                if(fil[momo]=='g'):
                    
                    b1_now=redshift_p*redshift_p*mapping_coeff.loc['g', 'm1']+mapping_coeff.loc['g', 'c1']*redshift_p+mapping_coeff.loc['g', 'c0']
                    b2_now=redshift_p*mapping_coeff.loc['g', 'm2']+mapping_coeff.loc['g', 'c2']
                        
                    sigma_b1_now=math.pow(mapping_coeff.loc['g', 'std_m1'],2)*math.pow(redshift_p,4)+math.pow(mapping_coeff.loc['g', 'std_c1'],2)*math.pow(redshift_p,2)+math.pow(mapping_coeff.loc['g', 'std_c0'],2)
                    sigma_b2_now=square(mapping_coeff.loc['g', 'std_m2']*redshift_p)+square(mapping_coeff.loc['g', 'std_c2'])
                    
                elif(fil[momo]=='r'):
                    b1_now=redshift_p*redshift_p*mapping_coeff.loc['r', 'm1']+mapping_coeff.loc['r', 'c1']*redshift_p+mapping_coeff.loc['r', 'c0']
                    b2_now=redshift_p*mapping_coeff.loc['r', 'm2']+mapping_coeff.loc['r', 'c2']

                    sigma_b1_now=math.pow(mapping_coeff.loc['r', 'std_m1'],2)*math.pow(redshift_p,4)+math.pow(mapping_coeff.loc['r', 'std_c1'],2)*math.pow(redshift_p,2)+math.pow(mapping_coeff.loc['r', 'std_c0'],2)
                    sigma_b2_now=square(mapping_coeff.loc['r', 'std_m2']*redshift_p)+square(mapping_coeff.loc['r', 'std_c2'])


                elif(fil[momo]=='i'):
                    b1_now=redshift_p*mapping_coeff.loc['i', 'm1']+mapping_coeff.loc['i', 'c1']
                    b2_now=redshift_p*mapping_coeff.loc['i', 'm2']+mapping_coeff.loc['i', 'c2'] 


                    sigma_b1_now=square(mapping_coeff.loc['i', 'std_m1']*redshift_p)+square(mapping_coeff.loc['i', 'std_c1'])
                    sigma_b2_now=square(mapping_coeff.loc['i', 'std_m2']*redshift_p)+square(mapping_coeff.loc['i', 'std_c2'])
                
                elif(fil[momo]=='z'):
                    b1_now=redshift_p*mapping_coeff.loc['z', 'm1']+mapping_coeff.loc['z', 'c1']
                    b2_now=redshift_p*mapping_coeff.loc['z', 'm2']+mapping_coeff.loc['z', 'c2'] 


                    sigma_b1_now=square(mapping_coeff.loc['z', 'std_m1']*redshift_p)+square(mapping_coeff.loc['z', 'std_c1'])
                    sigma_b2_now=square(mapping_coeff.loc['z', 'std_m2']*redshift_p)+square(mapping_coeff.loc['z', 'std_c2'])
                
                arr_pass=np.array([sqrt(sigma_b1_now), sqrt(sigma_b2_now)])
                priors=np.array([b1_now, b2_now])
                #print(array_mag[momo])
                    
                    
                if(momo==0):
                    #print(fil[momo])
                    chi1=np.zeros(tr)
                    final_params1=np.zeros((tr,4))
                    final_errors1=np.zeros((tr,4))
                    leng1=np.zeros(tr)

                    final_params=np.zeros(4)
                    final_errors=np.zeros(4)

                    t_cons1=(array_time[0])[np.argmin(array_mag[0])]
                    


                    for mmmm in range(tr):
                        # print('ini_params')
                        # print(ini_params)
                        ini_params=np.array([t_cons1+random.uniform(-3,+3)*(1+redshift_p), np.min(array_mag[momo]), b1_now+random.uniform(-2,2), b2_now+random.uniform(-2,2)])
                        # print(fil[momo])
                        # print(ini_params)
                        final_params1[mmmm,:], final_errors1[mmmm,:], chi1[mmmm], leng1[mmmm] = mpfit_run_flux1(array_time[momo], array_mag[momo], array_err[momo], ini_params, t_cons1-10*(1+redshift_p), t_cons1+10 *(1+redshift_p), arr_pass, priors, redshift_p)
                    
                    
                    # print('chi1')
                    # print(chi1)
                    min_ind=np.argmin(chi1)
                    final_params[:]=final_params1[min_ind,:]
                    final_errors[:]=final_errors1[min_ind,:]
                    chi=chi1[min_ind]
                    leng=leng1[min_ind]

 
                    if(chi<10000):
                        #print(redshift_p)
                        t_cons2=final_params[0]
                        
                        data_1.append([final_params[0], final_params[1], final_params[2], final_params[3],
                                        final_errors[0], final_errors[1], final_errors[2], final_errors[3], 
                                        chi, fil[momo],   redshift_p, redshift_e, leng, obj_id, ii, type])
                        
                        data_3.append([final_params[0], final_params[1], final_params[2], final_params[3],
                                        final_errors[0], final_errors[1], final_errors[2], final_errors[3], 
                                        chi, fil[momo],   redshift_p, redshift_e, leng,  obj_id, ii, type])

                    
                        
                        tmp_table = Table([array_time[momo], array_mag[momo], array_err[momo]],
                                        names=('time', 'mag', 'magerr'),
                                        meta={'redshift_p': redshift_p,
                                            'redshift_e': redshift_e, 'filter':fil[momo]})
                        tmp_table.write("tables/"+str(ii)+'_'+fil[momo]+'.csv', overwrite=True)
                        table_attm1.append(tmp_table)
                        table_attm3.append(tmp_table)


                    else:
                        break


                else:

                    chi1=np.zeros(tr)
                    final_params1=np.zeros((tr,4))
                    final_errors1=np.zeros((tr,4))
                    leng1=np.zeros(tr)

                    final_params=np.zeros(4)
                    final_errors=np.zeros(4)

                   
                    for mmmm in range(tr):
                        # print('ini_params')
                        # print(ini_params)
                        
                        ini_params=np.array([t_cons2+random.uniform(-2,2)*(1+redshift_p), np.min(array_mag[momo]), b1_now+random.uniform(-2,2), b2_now+random.uniform(-2,2)])
                        
                    
                        final_params1[mmmm,:], final_errors1[mmmm,:], chi1[mmmm], leng1[mmmm] = mpfit_run_flux1(array_time[momo], array_mag[momo], array_err[momo], ini_params, t_cons2-7*(1+redshift_p), t_cons2+7*(1+redshift_p), arr_pass, priors, redshift_p)
                       
                    # print('chi1')
                    # print(chi1)
                    min_ind=np.argmin(chi1)
                    final_params[:]=final_params1[min_ind,:]
                    final_errors[:]=final_errors1[min_ind,:]
                    chi=chi1[min_ind]
                    leng=leng1[min_ind]

                

                    if(chi<10000):
                        
                        data_1.append([final_params[0], final_params[1], final_params[2], final_params[3],
                                        final_errors[0], final_errors[1], final_errors[2], final_errors[3], 
                                        chi, fil[momo],  redshift_p, redshift_e, leng, obj_id, ii, type])
                        
                        data_3.append([final_params[0], final_params[1], final_params[2], final_params[3],
                                        final_errors[0], final_errors[1], final_errors[2], final_errors[3], 
                                        chi, fil[momo],  redshift_p, redshift_e, leng, obj_id, ii, type])
                        
                        tmp_table = Table([array_time[momo], array_mag[momo], array_err[momo]],
                                        names=('time', 'mag', 'magerr'),
                                        meta={ 'redshift': redshift_p,
                                            'redshift_e': redshift_e,  'filter':fil[momo]})
                        tmp_table.write("tables/"+str(ii)+'_'+fil[momo]+'.csv', overwrite=True)
                        table_attm1.append(tmp_table)
                        table_attm3.append(tmp_table)

            #print(data_1)
            data_2=np.array(data_1)
            df = pd.DataFrame(data_2)
            df.to_csv("mpfit_temp.csv")
            model1=pd.read_csv("mpfit_temp.csv")
            model=np.array(model1)
            model=np.delete(model, 0, axis=1)
            #print(len(model))

            legen=[]


            if(len(model)>0 ):

                for j in range(len(model)):
                
                    plo_data_x=eigen_val[:,0]*(1+model[j,10])+model[j,0]-model[0,0]
                    plo_data_y=model[j,1] + eigen_val[:,1] + model[j,2]*eigen_val[:,2] + model[j,3]*eigen_val[:,3]+dict_number[model[j,9]]
                    
                    # print('now')
                    # print(np.nanmin(plo_data_y[1:]))
                    # print(np.nanmax(plo_data_y[1:]))
                    
                   
                    plt.plot(plo_data_x, plo_data_y, color=dict_color[model[j,9]],   linestyle='solid', label='_nolegend_') 
                    
                    
                    plt.errorbar(table_attm1[j][0][:]-model[0,0],  table_attm1[j][1][:]+dict_number[model[j,9]], yerr=table_attm1[j][2][:], color=dict_color[model[j,9]], fmt='*', markersize=8, elinewidth=1.5, label=dict_legend[model[j,9]])
                    
                
                    coor=250
                    plt.annotate(dict_legend[model[j,9]], xy = (plo_data_x[coor], plo_data_y[coor]), xytext = (1*plo_data_x[coor], 0.99*plo_data_y[coor]), color  = dict_color[model[j,9]])
                    
                    plo_data_y=np.array(plo_data_y)
                    if(j==0):
                        ymin=np.nanmin(plo_data_y[1:])
                        ymax=np.nanmax(plo_data_y[1:])
                    else:
                        ymin=min(np.nanmin(plo_data_y[1:]), ymin)
                        ymax=max(np.nanmax(plo_data_y[1:]), ymax)
            
                plt.xlim([-10*(1+model[0,10])-7, +40*(1+model[0,10])+7])
                plt.ylim([ymin-0.5, ymax+0.5])
                
            
                plt.xlabel('Time (Days)')
                plt.ylabel('Magnitude')
                plt.gca().invert_yaxis()

    
        
                rw=np.round(model[j,10],2)
               
                plt.title('ID= '+str(obj_id)+'  '   + '  '      +'z= '+ str(rw) +'  '  +  '  type= '+ str(type) )
                   
                
                plt.savefig("plots/"+str(model[0,-2]) +'.png')
                plt.close()
               






data_4=np.array(data_3)
df = pd.DataFrame(data_4)
df.to_csv("des_good1.csv", header=['pk_time', 'pk_mag', 'alpha', 'beta', 'err_time', 'err_mag',
                                      'err_alpha', 'err_beta', 'chi', 'filter', 
                                     'photo_z', 'z_err', 'dof', 'obj_id', 'meta_id',  'type'], index=False)


df=pd.read_csv('des_good1.csv')


print(len(data_4))

print('done')


ii=0
ii=100
ii=200
ii=300
ii=400
ii=500
ii=600
ii=700
ii=800
ii=900
ii=1000
ii=1100
ii=1200
ii=1300
ii=1400
ii=1500
ii=1600
ii=1700
ii=1800
ii=1900
ii=2000
ii=2100
ii=2200
ii=2300
ii=2400
ii=2500
ii=2600
ii=2700
ii=2800
ii=2900
ii=3000
ii=3100
ii=3200
ii=3300
ii=3400
ii=3500
ii=3600
ii=3700
ii=3800
ii=3900
ii=4000
ii=4100
ii=4200
ii=4300
ii=4400
ii=4500
ii=4600
ii=4700
ii=4800
ii=4900
ii=5000
ii=5100
ii=5200
ii=5300
ii=5400
ii=5500
ii=5600
ii=5700
ii=5800
ii=5900
ii=6000
ii=6100
ii=6200
ii=6300
ii=6400
ii=6500
ii=6600
ii=6700
ii=6800
ii=6900
ii=7000
ii=7100
ii=7200
ii=7300
ii=7400
ii=7500
ii=7600
ii=7700
ii=7800
ii=7900
ii=8000
ii=8100
ii=8200
ii=8300
ii=8400
ii=8500
ii=8600
ii=8700
ii=8800
ii=8900
ii=9000
ii=9100
ii=9200
ii=9300
ii=9400
ii=9500
ii=9600
ii=9700
ii=9800
ii=9900
ii=10000
ii=10100
ii=10200
ii=10300
ii=10400
ii=10500
ii=10600
ii=10700
ii=10800
ii=10900
ii=11000
ii=11100
ii=11200
ii=11300
ii=11400
ii=11500
ii=11600
ii=11700
ii=11800
ii=11900
ii=12000
ii=12100
ii=12200
ii=12

In [26]:

df_long = pd.read_csv('des_good1.csv')



df_long['meta_id'] = df_long['meta_id'].astype(int)
df_long['filter'] = df_long['filter'].astype(str)

# --- 2. Define Columns for Pivoting and Metadata ---

# 8 columns per filter to be spread out
value_cols = [
    'pk_time', 'pk_mag', 'alpha', 'beta',
    'err_time', 'err_mag', 'err_alpha', 'err_beta',
]

# Columns to keep as object metadata (should be consistent across all filters for an object)
meta_cols = [
    'photo_z', 'z_err', 'obj_id', 'type'
]

# --- 3. Pivot the Data from Long to Wide Format ---

# Pivot on 'meta_id' to get one row per object, using 'filter' to spread the columns
pivoted_df = df_long.pivot(
    index='meta_id',
    columns='filter',
    values=value_cols
)

# --- 4. Flatten Multi-Index Columns and Rename ---

# Flatten the column names to the format 'filter_parameter' (e.g., 'g_alpha')
# The structure is (parameter_name, filter_name), so we reverse the order
pivoted_df.columns = [f'{col[1]}_{col[0]}' for col in pivoted_df.columns]
pivoted_df = pivoted_df.reset_index() # Move 'meta_id' from index back to a column

# --- 5. Merge Metadata and Final Output ---

# Extract unique metadata rows and merge them back with the pivoted data
meta_unique_df = df_long[['meta_id'] + meta_cols].drop_duplicates(subset=['meta_id']).reset_index(drop=True)

# Merge the metadata onto the wide-format results
final_wide_df = meta_unique_df.merge(pivoted_df, on='meta_id', how='right')

# 6. Save the final wide-format CSV file
output_filename = 'des_good_coverage.csv'
final_wide_df.to_csv(output_filename, index=False)

print(f"\n✅ Successfully created the wide-format file: {output_filename}")
print(f"Final DataFrame has {len(final_wide_df)} rows (one per object).")

print(len(final_wide_df))


✅ Successfully created the wide-format file: des_good_coverage.csv
Final DataFrame has 456 rows (one per object).
456
